In [1]:
import pandas as pd
import numpy as np

In [2]:
def round_hour(row, variable):
    if int(row[variable + '_min']) > 30:
        return int(row[variable +'_hour']) + 1
    else:
        return int(row[variable +'_hour'])

In [3]:
def get_cyclical_time(df, variable):
    df[variable]= df[variable].replace([2400],0)
    df[variable] = df[variable].astype(int).astype(str).str.zfill(4) 
    df[variable] = pd.to_datetime(df[variable], format='%H%M')
    df[variable +'_hour'] = df[variable].dt.hour
    df[variable + '_min'] = df[variable].dt.minute
    df[variable +'_hour'] = df.apply(lambda x: round_hour(x, variable), axis=1)
    df['sin_' + variable] = np.sin(df[variable +'_hour'])
    df['cos_' + variable] = np.cos(df[variable +'_hour'])

In [4]:
def is_delayed(row):
    if row['arr_delay'] <= 15:
        return 0
    else:
        return 1

def how_delayed(row):
    if row['arr_delay'] <= 15:
        return 0
    else:
        return row['arr_delay']

In [5]:
def feat(df):
    df['fl_date'] = pd.to_datetime(df['fl_date'])
    df['date']= df['fl_date'].dt.day
    df['weekday'] = df['fl_date'].dt.dayofweek
    df['sin_weekday'] = np.sin(df['weekday'])
    df['cos_weekday'] = np.cos(df['weekday'])

In [9]:
def get_features(path, path2, csv):
    dftrain = pd.read_csv(path + csv)
    dftest = pd.read_csv(path2 + csv)
    dfpred_test = pd.read_csv(path2 + 'cleanedtest_finalraw_flights_test.csv')
    feat(dftrain)
    feat(dftest)
    feat(dfpred_test)                        
    get_cyclical_time(dftrain, 'crs_dep_time')
    get_cyclical_time(dftest, 'crs_dep_time')
    get_cyclical_time(dfpred_test, 'crs_dep_time')
    get_cyclical_time(dftrain, 'dep_time')
    get_cyclical_time(dftrain, 'crs_arr_time')
    get_cyclical_time(dftest, 'crs_arr_time')
    get_cyclical_time(dfpred_test, 'crs_arr_time')
    get_cyclical_time(dftrain, 'arr_time')
    dftrain['delayed'] = dftrain.apply(lambda x: is_delayed(x), axis=1)
    dftrain['delay'] = dftrain.apply(lambda x: how_delayed(x), axis=1)
    # average departure delay and number of fligths per carrier
    dftrain = dftrain.join(dftrain.groupby('op_unique_carrier').mean()['dep_delay'], on='op_unique_carrier', rsuffix='A').rename({'dep_delayA': 'DelayD_carrier'}, axis=1)
    dftrain = dftrain.join(dftrain.groupby('op_unique_carrier').count()['fl_date'], on='op_unique_carrier', rsuffix='A').rename({'fl_dateA': 'num_flights_carrier'}, axis=1)
    dftest = dftest.join(dftrain.groupby('op_unique_carrier').mean()['dep_delay'], on='op_unique_carrier', rsuffix='A').rename({'dep_delayA': 'DelayD_carrier'}, axis=1)
    dftest = dftest.join(dftest.groupby('op_unique_carrier').count()['fl_date'], on='op_unique_carrier', rsuffix='A').rename({'fl_dateA': 'num_flights_carrier'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby('op_unique_carrier').mean()['dep_delay'], on='op_unique_carrier', rsuffix='A').rename({'dep_delayA': 'DelayD_carrier'}, axis=1)
    dfpred_test = dfpred_test.join(dfpred_test.groupby('op_unique_carrier').count()['fl_date'], on='op_unique_carrier', rsuffix='A').rename({'fl_dateA': 'num_flights_carrier'}, axis=1)
    # average departure delay and number of flights per origin
    dftrain = dftrain.join(dftrain.groupby('origin').mean()['dep_delay'], on='origin', rsuffix='A').rename({'dep_delayA': 'DelayD_origin'}, axis=1)
    dftrain = dftrain.join(dftrain.groupby('origin').count()['fl_date'], on='origin', rsuffix='A').rename({'fl_dateA': 'num_flights_origin'}, axis=1)
    dftest = dftest.join(dftrain.groupby('origin').mean()['dep_delay'], on='origin', rsuffix='A').rename({'dep_delayA': 'DelayD_origin'}, axis=1)
    dftest = dftest.join(dftest.groupby('origin').count()['fl_date'], on='origin', rsuffix='A').rename({'fl_dateA': 'num_flights_origin'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby('origin').mean()['dep_delay'], on='origin', rsuffix='A').rename({'dep_delayA': 'DelayD_origin'}, axis=1)
    dfpred_test = dfpred_test.join(dftest.groupby('origin').count()['fl_date'], on='origin', rsuffix='A').rename({'fl_dateA': 'num_flights_origin'}, axis=1)
    # average departure delay and number of flights per destination
    dftrain = dftrain.join(dftrain.groupby('dest').mean()['dep_delay'], on='dest', rsuffix='A').rename({'dep_delayA': 'DelayD_dest'}, axis=1)
    dftrain = dftrain.join(dftrain.groupby('dest').count()['fl_date'], on='dest', rsuffix='A').rename({'fl_dateA': 'num_flights_dest'}, axis=1)
    dftest = dftest.join(dftrain.groupby('dest').mean()['dep_delay'], on='dest', rsuffix='A').rename({'dep_delayA': 'DelayD_dest'}, axis=1)
    dftest = dftest.join(dftest.groupby('dest').count()['fl_date'], on='dest', rsuffix='A').rename({'fl_dateA': 'num_flights_dest'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby('dest').mean()['dep_delay'], on='dest', rsuffix='A').rename({'dep_delayA': 'DelayD_dest'}, axis=1)
    dfpred_test = dfpred_test.join(dfpred_test.groupby('dest').count()['fl_date'], on='dest', rsuffix='A').rename({'fl_dateA': 'num_flights_dest'}, axis=1)
    # average arrival delay per carrier, origin and destination
    dftrain = dftrain.join(dftrain.groupby('op_unique_carrier').mean()['arr_delay'], on='op_unique_carrier', rsuffix='A').rename({'arr_delayA': 'DelayA_carrier'}, axis=1)
    dftrain = dftrain.join(dftrain.groupby('origin').mean()['arr_delay'], on='origin', rsuffix='A').rename({'arr_delayA': 'DelayA_origin'}, axis=1)
    dftrain = dftrain.join(dftrain.groupby('dest').mean()['arr_delay'], on='dest', rsuffix='A').rename({'arr_delayA': 'DelayA_dest'}, axis=1)
    dftest = dftest.join(dftrain.groupby('op_unique_carrier').mean()['arr_delay'], on='op_unique_carrier', rsuffix='A').rename({'arr_delayA': 'DelayA_carrier'}, axis=1)
    dftest = dftest.join(dftrain.groupby('origin').mean()['arr_delay'], on='origin', rsuffix='A').rename({'arr_delayA': 'DelayA_origin'}, axis=1)
    dftest = dftest.join(dftrain.groupby('dest').mean()['arr_delay'], on='dest', rsuffix='A').rename({'arr_delayA': 'DelayA_dest'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby('op_unique_carrier').mean()['arr_delay'], on='op_unique_carrier', rsuffix='A').rename({'arr_delayA': 'DelayA_carrier'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby('origin').mean()['arr_delay'], on='origin', rsuffix='A').rename({'arr_delayA': 'DelayA_origin'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby('dest').mean()['arr_delay'], on='dest', rsuffix='A').rename({'arr_delayA': 'DelayA_dest'}, axis=1)
    # average arrival delay and number of flights per hour(Dep)
    dftrain = dftrain.join(dftrain.groupby('crs_dep_time_hour').mean()['arr_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'arr_delayA': 'DelayA_hourly'}, axis=1)
    dftrain = dftrain.join(dftrain.groupby('crs_dep_time_hour').count()['arr_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'arr_delayA': 'num_flights_hourly'}, axis=1)
    dftest = dftest.join(dftrain.groupby('crs_dep_time_hour').mean()['arr_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'arr_delayA': 'DelayA_hourly'}, axis=1)
    dftest = dftest.join(dftest.groupby('crs_dep_time_hour').count()['fl_date'], on='crs_dep_time_hour', rsuffix='A').rename({'fl_dateA': 'num_flights_hourly'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby('crs_dep_time_hour').mean()['arr_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'arr_delayA': 'DelayA_hourly'}, axis=1)
    dfpred_test = dfpred_test.join(dfpred_test.groupby('crs_dep_time_hour').count()['fl_date'], on='crs_dep_time_hour', rsuffix='A').rename({'fl_dateA': 'num_flights_hourly'}, axis=1)
    # average departure delay per hour(Dep)
    dftrain = dftrain.join(dftrain.groupby('crs_dep_time_hour').mean()['dep_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'dep_delayA': 'DelayD_hourly'}, axis=1)
    dftest = dftest.join(dftrain.groupby('crs_dep_time_hour').mean()['dep_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'dep_delayA': 'DelayD_hourly'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby('crs_dep_time_hour').mean()['dep_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'dep_delayA': 'DelayD_hourly'}, axis=1)
    # average arrival delay per hour (per origin and per carrier)
    dftrain = dftrain.join(dftrain.groupby(['crs_dep_time_hour', 'origin']).mean()['arr_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'arr_delayA': 'DelayA_origin_hourly'}, axis=1)
    dftrain = dftrain.join(dftrain.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['arr_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'arr_delayA': 'DelayA_carrier_hourly'}, axis=1)
    dftest = dftest.join(dftrain.groupby(['crs_dep_time_hour', 'origin']).mean()['arr_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'arr_delayA': 'DelayA_origin_hourly'}, axis=1)
    dftest = dftest.join(dftrain.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['arr_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'arr_delayA': 'DelayA_carrier_hourly'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby(['crs_dep_time_hour', 'origin']).mean()['arr_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'arr_delayA': 'DelayA_origin_hourly'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['arr_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'arr_delayA': 'DelayA_carrier_hourly'}, axis=1)
    # average arrival delay per hour (per origin and per carrier)
    dftrain = dftrain.join(dftrain.groupby(['crs_dep_time_hour', 'origin']).mean()['dep_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'dep_delayA': 'DelayD_origin_hourly'}, axis=1)
    dftrain = dftrain.join(dftrain.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['dep_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'dep_delayA': 'DelayD_carrier_hourly'}, axis=1)
    dftest = dftest.join(dftrain.groupby(['crs_dep_time_hour', 'origin']).mean()['dep_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'dep_delayA': 'DelayD_origin_hourly'}, axis=1)
    dftest = dftest.join(dftrain.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['dep_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'dep_delayA': 'DelayD_carrier_hourly'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby(['crs_dep_time_hour', 'origin']).mean()['dep_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'dep_delayA': 'DelayD_origin_hourly'}, axis=1)
    dfpred_test = dfpred_test.join(dftrain.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['dep_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'dep_delayA': 'DelayD_carrier_hourly'}, axis=1)
    dftrain.to_csv(f'{path}engineered_{csv}', index=None)
    dftest.to_csv(f'{path2}engineered_test{csv}', index=None)
    dfpred_test.to_csv(f'{path2}engineered_predtest{csv}', index=None)

In [10]:
path = 'final2/Training/'
path2 = 'final2/Validation/'
file1 = 'cleanedtest_finalraw_flights_Dec_Jan.csv'
file2 = 'cleanedtest_finalraw_flights_Dec.csv' 
file3 = 'cleanedtest_finalraw_flights_Jan_1w.csv' 
file4 = 'cleanedtest_finalraw_flights_Jan_2w.csv' 
file5 = 'cleanedtest_finalraw_flights_Jan.csv' 
files = [file1,file2,file3,file4,file5]

In [ ]:
for i in files:
    get_features(path, path2, i)